In [1]:
import numpy as np
import pandas as pd

In [16]:
# 加总投入产出表

## 导入原始数据
full_table = pd.read_excel('./China Multi-Regional Input-Output Table 2012.xlsx', sheet_name='Table')
full_IO = np.array(full_table.iloc[3:1305, 3:1305].reset_index(drop=True))

## 汇总至62×62的投入产出表
N = 31
J = 2

aux_IO = np.ndarray([N*J, 1302])
for i in range(N*J): ## 先列向加总
    if i % 2 == 0:
        aux_IO[i, :] = full_IO[i//2*42, :]
    if i % 2 == 1:
        aux_IO[i, :] = full_IO[i//2*42+1 : (i//2+1)*42, :].sum(axis=0)

IO = np.ndarray([N*J, N*J])
for i in range(N*J): ## 再行向加总
    if i % 2 == 0:
        IO[:, i] = aux_IO[:, i//2*42]
    if i % 2 == 1:
        IO[:, i] = aux_IO[:, i//2*42+1 : (i//2+1)*42].sum(axis=1)

In [17]:
# 构建贸易份额

## 在投入产出表中去掉西藏
IO = np.delete(IO, [50, 51], axis=0) # 删除西藏
IO = np.delete(IO, [50, 51], axis=1) # 删除西藏

## 计算贸易份额矩阵
trade_share = IO / IO.sum(axis = 0).reshape([1, 60])

prov = list(full_table.iloc[:, 0].dropna())
prov.remove('西藏')
prov_sector = []
for each_prov in prov:
    prov_sector.append(each_prov + '-农业部门')
    prov_sector.append(each_prov + '-非农部门')

trade_share  = pd.DataFrame(trade_share)
trade_share.index, trade_share.columns = prov_sector, prov_sector
trade_share.to_excel('./trade_share.xlsx')

In [18]:
## 构建只考虑相同部门间贸易的贸易份额

trade_share = np.array(pd.read_excel('./trade_share.xlsx').iloc[:, 1:])
for column in range(60):
    if column % 2 == 0:
        for index in range(1, 60, 2):
            trade_share[index, column] = 0
    if column % 2 == 1:
        for index in range(0, 60, 2):
            trade_share[index, column] = 0

trade_share = (trade_share.T / trade_share.sum(axis=0).reshape([60, 1])).T

trade_share  = pd.DataFrame(trade_share)
trade_share.index, trade_share.columns = prov_sector, prov_sector
trade_share.to_excel('./trade_share1.xlsx')